In [106]:
import pandas as pd 
import os
import psycopg2
import numpy as np
import psycopg2.extras as extras
import string
import random
from sklearn.preprocessing import MultiLabelBinarizer

## Load airbnb dataset 

In [107]:
airbnb = pd.read_csv('analysisData.csv')
airbnb.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (31,79) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,name,summary,space,description,neighborhood_overview,notes,transit,access,interaction,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5935333,<Williamsburg Apartment>,My place is close to L train and walking dista...,NaN,My place is close to L train and walking dista...,NaN,NaN,NaN,NaN,NaN,...,f,f,strict_14_with_grace_period,f,f,1,1,0,0,0.03
1,16656693,<Large 1 bedroom and private bath in Astoria>,"Located in the Astoria Arts District, this 1 ...","The bedroom includes a twin bed, a TV, dress...","Located in the Astoria Arts District, this 1 ...","Located in the Astoria Arts District, you’ll ...",NaN,Best way to get around is the subway. There’s...,Guests are welcome to use the full kitchen and...,I’m also happy to share a glass of wine with g...,...,t,f,strict_14_with_grace_period,f,f,1,0,1,0,0.61
2,19884380,<SPACIOUS ROOM 19 MINUTES TO TIME SQUARE>,An american Bedroom with Newyorker stile. 1 Q...,This is an incredible room with 2 windows givi...,An american Bedroom with Newyorker stile. 1 Q...,15 minutes to Central Park 20 to time Square ...,NaN,The number ! train is 2 blocks way from the ap...,The kitchen is ready for you to prepare any Me...,I’m always in contact with the guest by phone ...,...,f,f,strict_14_with_grace_period,f,f,4,0,4,0,3.56
3,35928387,<True loft rental>,Comfortable and cozy space over looking the ci...,Duplex bright loft with everything you need!,Comfortable and cozy space over looking the ci...,NaN,We do have more past reviews available for a ...,5 minutes to the L train at the Morgan stop. ...,The loft and the rooftop;),NaN,...,f,f,moderate,t,t,1,1,0,0,0.04
4,16763347,<Upper East Side Getaway! Lots of space!>,"LARGE common room, TV, couch, chairs, musi...",NaN,"LARGE common room, TV, couch, chairs, musi...",NaN,NaN,NaN,NaN,NaN,...,f,f,flexible,f,f,1,1,0,0,0.72


In [108]:
airbnb.shape

(39527, 91)

## Connect to the Postgresql

In [109]:
param_dic = {
    "host"      : "localhost",
    "port"      : "5432",
    "database"  : "airbnb",
    "user"      : "postgres",
    "password"  : "123"
}

In [110]:
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn
conn = connect(param_dic)

Connecting to the PostgreSQL database...
Connection successful


In [111]:
from sqlalchemy import create_engine

connect = "postgresql+psycopg2://%s:%s@%s:5432/%s" % (
    param_dic['user'],
    param_dic['password'],
    param_dic['host'],
    param_dic['database']
)

def to_alchemy(df, name):
    """
    Using a dummy table to test this call library
    """
    engine = create_engine(connect)
    df.to_sql(
        name, 
        con=engine, 
        index=False, 
        if_exists='append'
    )
    print("to_sql() done (sqlalchemy)")

## Create and transform 15 tables to the postgresql

In [26]:
cur = conn.cursor()
createCmd = """ 


create table population(
borough_id varchar(10) primary key,
borough_name varchar(50) not null,
population_2020 bigint,
population_share_of_NYC_in_2020 numeric
);

create table property(
property_id varchar(10) primary key,
property_type varchar(45)
 );


create table room (
room_id varchar(10) primary key,
room_type varchar(100)
);

create table bed_type (
bed_id varchar(10) primary key,
type varchar(100)
);

create table neighbourhood (
neighbour_id varchar(10) primary key,
borough_id varchar(10),
neighbourhood varchar(100)
);


create table house(
house_id varchar(10) primary key,
bedrooms int,
beds int,
price numeric,
accommodates int
);

create table house_info (
house_id varchar(10) primary key,
property_id varchar(10),
room_id varchar(10),
borough_id varchar(10),
bed_id varchar (10),
neighbour_id varchar(10),
foreign key (property_id) references property (property_id),
foreign key (room_id) references room (room_id),
foreign key (borough_id) references population (borough_id),
foreign key (bed_id) references bed_type(bed_id),
foreign key (neighbour_id) references neighbourhood(neighbour_id)
);


create table review(
review_id varchar(10) references house(house_id),
number_of_reviews int,
review_scores_value int
);

create table host(
house_id varchar(10) references house(house_id),
host_name varchar(200),
host_since date,
host_is_superhost varchar(5)
);

create table policy(
policy_id varchar(10) primary key,
instant_bookable varchar(5),
cancellation_policy varchar(255)
);

create table house_policy(
house_id varchar(10),
policy_id varchar (10),
primary key (house_id, policy_id),
constraint fk_house foreign key (house_id) references house (house_id),
constraint fk_policy foreign key (policy_id) references policy (policy_id)
);

create table amenity(
amenity_id varchar(10) references house (house_id),
Wifi boolean,
Heating boolean,
TV boolean,
Kitchen boolean
);

create table parks(
park_id varchar(10) primary key,
borough_id varchar(10),
park_type varchar(50),
foreign key (borough_id) references population (borough_id)
);

create table shooting_incident(
incident_id varchar(10) primary key,
borough_id varchar(10),
occur_date date,
murder varchar(5),
foreign key (borough_id) references population (borough_id)
);

create table maintenance_fee(
fee_id varchar(10) primary key,
borough_id varchar(50),
avg_amount int,
foreign key (borough_id) references population (borough_id)
);
"""
    
cur.execute(createCmd)
conn.commit()


## Subset dataset

In [10]:
airbnb.columns

Index(['id', 'name', 'summary', 'space', 'description',
       'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
       'house_rules', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'is_location_exact',
       'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'bed_type', 'amenities', 'square_feet', 'price', 'weekly_price',
       'monthly_price', 'security_deposit', 'cleaning_fee', 'guests_included',
       'extra_people', 'minimum_nights', 'maximum_nights',
       'minimum_mi

### Data cleanning and transforming values to each table in postgresql

#### property table 

In [30]:
Property = airbnb[["property_type"]]
Property = Property.drop_duplicates().reset_index(drop=True)
Property["property_id"] = random.sample(range(1000,9999), Property.shape[0])

In [31]:
to_alchemy(Property, "property")

to_sql() done (sqlalchemy)


#### room table

In [32]:
room = airbnb[["room_type"]]
room = room.drop_duplicates().reset_index(drop=True)
room["room_id"] = random.sample(range(1000,9999), room.shape[0])


In [33]:
to_alchemy(room, "room")

to_sql() done (sqlalchemy)


#### population table

In [34]:
population = pd.read_csv("New_York_City_Population_by_Borough__1950_-_2040.csv")
population = population[["Borough","2020","2020 - Boro share of NYC total"]]
population = population.rename(columns={"Borough":"borough_name",
                           "2020":"population_2020",
                           "2020 - Boro share of NYC total":"population_share_of_nyc_in_2020"})
population["borough_name"] = population["borough_name"].str.strip()
population["borough_id"] = random.sample(range(1000,9999),population.shape[0])
population = population.iloc[1:,:]

In [35]:
population 

,borough_name,population_2020,population_share_of_nyc_in_2020,borough_id
1,Bronx,1446788,16.92,1077
2,Brooklyn,2648452,30.97,8656
3,Manhattan,1638281,19.16,7332
4,Queens,2330295,27.25,2870
5,Staten Island,487155,5.70,9331


In [36]:
to_alchemy(population, "population")

to_sql() done (sqlalchemy)


#### bed_type table

In [37]:
bed_type = airbnb[["bed_type"]]
bed_type = bed_type.drop_duplicates().reset_index(drop=True)
bed_type["bed_id"] = random.sample(range(1000,9999), bed_type.shape[0])
bed_type = bed_type.rename(columns={"bed_type":"type"})

In [38]:
to_alchemy(bed_type,"bed_type")

to_sql() done (sqlalchemy)


#### neighborhood table

In [39]:
neighbourhood = pd.read_csv('/Users/zhaoyilu/Desktop/5310 sql project/neighbourhoods.csv')
neighbourhood = pd.merge(neighbourhood, population, how='left', left_on=["neighbourhood_group"],
                       right_on=["borough_name"])
neighbourhood = neighbourhood[["borough_name", "neighbourhood","borough_id"]]
neighbourhood["neighbour_id"] = random.sample(range(1000,9999), neighbourhood.shape[0])
neighbourhood = neighbourhood[["neighbourhood","borough_id","neighbour_id"]]

In [40]:
to_alchemy(neighbourhood,"neighbourhood")

to_sql() done (sqlalchemy)


#### house table

In [41]:
house = airbnb[["id","property_type","room_type","neighbourhood_group_cleansed","bedrooms","beds",
                "price","accommodates","bed_type","neighbourhood_cleansed"]]
house = pd.merge(house, Property, how="left",left_on=["property_type"], right_on=["property_type"])
house = pd.merge(house, room, how="left",left_on=["room_type"], right_on=["room_type"])
house = pd.merge(house, population, how="left",left_on=["neighbourhood_group_cleansed"],
                right_on=["borough_name"])
house = pd.merge(house, bed_type, how="left",left_on=["bed_type"],right_on=["type"])
house = pd.merge(house, neighbourhood[["neighbour_id","neighbourhood"]], how='left', left_on=["neighbourhood_cleansed"],
                right_on=["neighbourhood"])
house = house[["id","property_id","room_id","borough_id","neighbour_id","bedrooms",
              "beds","bed_id","price","accommodates"]]
house = house.rename(columns={"id":"house_id"})

#### house_info table

In [42]:
house_info = house[["house_id","property_id","room_id","borough_id","neighbour_id","bed_id"]]

In [43]:
to_alchemy(house_info,"house_info")

to_sql() done (sqlalchemy)


#### transform house table value 

In [44]:
house = house[["house_id", "bedrooms","beds","price","accommodates"]]

In [45]:
to_alchemy(house,"house")

to_sql() done (sqlalchemy)


#### review table

In [46]:
review = airbnb[["id","number_of_reviews","review_scores_value"]]
review = review.rename(columns={"id":"review_id"})

In [47]:
to_alchemy(review,"review")

to_sql() done (sqlalchemy)


#### host table

In [48]:
host = airbnb[["id","host_name","host_since","host_is_superhost"]]
host = host.rename(columns={"id":"house_id"})

In [49]:
to_alchemy(host,"host")

to_sql() done (sqlalchemy)


#### policy table

In [50]:
policy = airbnb[["instant_bookable","cancellation_policy"]]
policy = policy.drop_duplicates().reset_index(drop=True)
policy["policy_id"] = random.sample(range(1000,9999),policy.shape[0])

In [51]:
to_alchemy (policy,"policy")

to_sql() done (sqlalchemy)


#### house_policy table

In [52]:
house_policy = airbnb[["id","instant_bookable","cancellation_policy"]]
house_policy = pd.merge(house_policy, policy, how="left",
                        left_on=["instant_bookable","cancellation_policy"],
                       right_on=["instant_bookable","cancellation_policy"])
house_policy = house_policy[["id","policy_id"]]
house_policy = house_policy.rename(columns={"id":"house_id"})

In [53]:
to_alchemy(house_policy,"house_policy")

to_sql() done (sqlalchemy)


#### amenity table

In [54]:
amenity = airbnb[["id","amenities"]]
amenity["amenities"] = amenity["amenities"].apply(lambda x: str(x).split(","))
amenity_list = ["Wifi","Heating","TV","Kitchen"]

def clean_text(l):
    for i in range(len(l)):
        l[i] = l[i].strip()
        l[i] = l[i].replace(".","")
        l[i] = l[i].replace(" ","")
    temp = []
    for j in l:
        if j in amenity_list:
            temp.append(j)
    return list(set(temp))

amenity["amenities"] = amenity["amenities"].apply(lambda x: clean_text(x))
mlb = MultiLabelBinarizer()
temp = pd.DataFrame(mlb.fit_transform(amenity.amenities),columns=mlb.classes_)
temp = temp == 1
amenity = pd.concat([amenity, temp],axis=1)
amenity = amenity.drop(["amenities"],axis=1)
amenity = amenity.rename(columns = {"id":"amenity_id"})
amenity = amenity.rename(columns = {"Wifi":"wifi","Heating":"heating","TV":"tv","Kitchen":"kitchen"})

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [55]:
amenity

,amenity_id,heating,kitchen,tv,wifi
0,5935333,True,True,True,True
1,16656693,True,True,True,True
2,19884380,True,True,True,True
3,35928387,True,True,True,True
4,16763347,True,True,True,True
...,...,...,...,...,...
39522,17394978,True,False,True,True
39523,17636257,True,False,False,True
39524,19560004,True,True,False,True
39525,6459378,True,True,True,True


In [56]:
to_alchemy(amenity,"amenity")

to_sql() done (sqlalchemy)


#### parks table

In [57]:
parks = pd.read_csv("Parks_Properties.csv")
parks = parks[["BOROUGH","TYPECATEGORY"]]
parks["BOROUGH"] = parks["BOROUGH"].map({"B":"Brooklyn","Q":"Queens","X":"Bronx",
                                         "M":"Manhattan","R":"Staten Island"})
parks = parks.rename(columns={"BOROUGH":"borough_name","TYPECATEGORY":"park_type"})
parks = pd.merge(parks, population[["borough_name","borough_id"]], how='left', 
                 left_on=["borough_name"], right_on=["borough_name"])
parks["park_id"] = random.sample(range(1000,9999),parks.shape[0])
parks = parks.drop(["borough_name"],axis=1)

In [58]:
to_alchemy(parks,"parks")

to_sql() done (sqlalchemy)


#### shooting_incident table

In [59]:
shooting_incident = pd.read_csv("NYPD Shooting Incident - Data 2006-2021.csv")
shooting_incident = shooting_incident[["OCCUR_DATE","STATISTICAL_MURDER_FLAG","BORO"]]
shooting_incident["BORO"] = shooting_incident.BORO.str.title()
shooting_incident = pd.merge(shooting_incident, population[["borough_name","borough_id"]],
                            how='left', left_on=["BORO"], right_on=["borough_name"])
shooting_incident = shooting_incident[["OCCUR_DATE", "STATISTICAL_MURDER_FLAG", "borough_id"]]
shooting_incident = shooting_incident.rename(columns={"OCCUR_DATE":"occur_date",
                                                      "STATISTICAL_MURDER_FLAG":"murder"})
shooting_incident["incident_id"] = random.sample(range(1000,99999), shooting_incident.shape[0])

In [60]:
to_alchemy(shooting_incident,"shooting_incident")

to_sql() done (sqlalchemy)


#### maintenance_fee table

In [61]:
maintenance_fee = pd.read_csv("Fee_Charges.csv")
maintenance_fee = maintenance_fee[["Boro","FeeAmount"]]
maintenance_fee = pd.DataFrame(maintenance_fee.groupby("Boro")["FeeAmount"].mean()).reset_index()
maintenance_fee = maintenance_fee.rename(columns={"Boro":"borough_name","FeeAmount":"avg_amount"})
maintenance_fee["avg_amount"] = maintenance_fee["avg_amount"].astype("int")
maintenance_fee["borough_name"] = maintenance_fee["borough_name"].str.title()
maintenance_fee = pd.merge(maintenance_fee, population[["borough_name","borough_id"]],
                          how="left",left_on=["borough_name"], right_on=["borough_name"])
maintenance_fee = maintenance_fee[["borough_id", "avg_amount"]]
maintenance_fee["fee_id"] = random.sample(range(1000,9999),maintenance_fee.shape[0])

In [62]:
to_alchemy(maintenance_fee,"maintenance_fee")

to_sql() done (sqlalchemy)


### 10 questions 

#### 1. What’s the relationship between shooting_incident and rental price in each borough? 


In [74]:
# Establish a connection
cur = conn.cursor()
view1 = """
create view incident_borough as
select po.borough_id, po.borough_name, count(s.murder) as incident_number 
from population po join shooting_incident s on po.borough_id = s.borough_id 
group by po.borough_id;

create view house_type as 
select HI.borough_id, avg(h.price) as avg_price
from house h join house_info HI on h.house_id = HI.house_id
join property p on HI.property_id = p.property_id 
group by HI.borough_id;


"""
cur.execute(view1)
conn.commit()

In [75]:
# Execute the statement and get the results:
cur.execute(
    """ 
       select ht.avg_price, ib.incident_number, ib.borough_name 
        from house_type ht join incident_borough ib 
        on ht.borough_id = ib.borough_id
    """
)

rows = cur.fetchall()
colname = [desc[0] for desc in cur.description]

df=pd.DataFrame(rows, columns=colname)

df


,avg_price,incident_number,borough_name
0,174.7414802065404475,2974,Manhattan
1,85.0844155844155844,703,Staten Island
2,115.4558475526217462,9826,Brooklyn
3,75.1549295774647887,6794,Bronx
4,90.7379367720465890,3568,Queens


#### 2. What is the average price of each property type?

In [76]:
cur.execute(
    """ 
      select avg(h.price) as avg_price, p.property_type
      from house h join house_info HI on h.house_id = HI.house_id
      join property p on HI.property_id = p.property_id 
      group by p.property_type
      order by avg_price desc

    """
)

rows = cur.fetchall()
colname = [desc[0] for desc in cur.description]

df2=pd.DataFrame(rows, columns=colname)

df2


,avg_price,property_type
0,750.0000000000000000,Lighthouse
1,493.7560975609756098,Resort
2,425.0000000000000000,Timeshare
3,350.0000000000000000,Tent
4,325.0000000000000000,Farm stay
5,280.0000000000000000,Houseboat
6,207.3828756058158320,Loft
7,205.9512195121951220,Serviced apartment
8,204.7424242424242424,Hotel
9,197.9333333333333333,Boutique hotel


#### 3. What is the top 10 property type that has the highest price ?

In [77]:
cur = conn.cursor()
view2 = """
create view pp_rank as 
select p.property_type, h.price, 
dense_rank() over (partition by property_type order by price desc)
from house h join house_info HI on h.house_id = HI.house_id
join property p on HI.property_id = p.property_id;


"""
cur.execute(view2)
conn.commit()

In [78]:
cur.execute(
    """ 
      select property_type, price from pp_rank
      where dense_rank <= 1
      group by property_type,price
      order by price desc
      limit 10

    """
)

rows = cur.fetchall()
colname = [desc[0] for desc in cur.description]

df3=pd.DataFrame(rows, columns=colname)

df3


,property_type,price
0,Apartment,999
1,Resort,999
2,Boutique hotel,999
3,Loft,999
4,House,990
5,Condominium,975
6,Townhouse,950
7,Serviced apartment,890
8,Villa,890
9,Guesthouse,800


#### 4. What is the top 10 neighborhood that has the highest price of an Airbnb house in each borough?

In [79]:
cur = conn.cursor()
view3 = """
create view np_rank as 
select p.borough_name, n.neighbourhood, h.price, 
dense_rank() over (partition by p.borough_name order by n.neighbourhood, price desc)
from population p join house_info hi on p.borough_id = hi.borough_id
join house h on hi.house_id = h.house_id
join neighbourhood n on n.neighbour_id = hi.neighbour_id;


"""
cur.execute(view3)
conn.commit()

In [81]:
cur.execute(
    """ 
      select borough_name, neighbourhood, price from np_rank
      where dense_rank <=3
      group by neighbourhood, price,borough_name
      order by price desc
      limit 10

    """
)

rows = cur.fetchall()
colname = [desc[0] for desc in cur.description]

df4=pd.DataFrame(rows, columns=colname)

df4

,borough_name,neighbourhood,price
0,Staten Island,Arrochar,625
1,Manhattan,Battery Park City,600
2,Queens,Arverne,550
3,Queens,Arverne,500
4,Queens,Arverne,450
5,Manhattan,Battery Park City,350
6,Manhattan,Battery Park City,325
7,Bronx,Allerton,186
8,Brooklyn,Bath Beach,180
9,Brooklyn,Bath Beach,166


#### 5. What is the correlation between rental price and maintenance fee?


In [82]:
cur.execute(
    """ 
      select corr(rental_price, maintenance_fee)
      from(
      select ht.avg_price rental_price, mf.avg_amount maintenance_fee
      from house_type ht join maintenance_fee mf
      on ht.borough_id = mf.borough_id) htmf

    """
)

rows = cur.fetchall()
colname = [desc[0] for desc in cur.description]

df5=pd.DataFrame(rows, columns=colname)

df5

,corr
0,0.461697


#### 6. Which borough in the NYC has the highest number of houses available for rental on Airbnb？

In [118]:
cur.execute(
    """ 
      select p.borough_name, count(h.house_id) as total_number_houses
      from population as p
      join house_info as h using (borough_id)
      group by p.borough_name
      order by total_number_houses DESC

    """
)

rows = cur.fetchall()
colname = [desc[0] for desc in cur.description]

df6=pd.DataFrame(rows, columns=colname)

df6

,borough_name,total_number_houses
0,Manhattan,17430
1,Brooklyn,16058
2,Queens,4808
3,Bronx,923
4,Staten Island,308


#### 7. Is there any relationship between rental price and neighborhood(rental price by neighbourhood)?


In [117]:
cur.execute(
    """ 
      select neighbourhood, avg(price) as avg_price
      from neighbourhood 
      join house_info using (neighbour_id)
      join house using (house_id)
      group by neighbourhood


    """
)

rows = cur.fetchall()
colname = [desc[0] for desc in cur.description]

df7=pd.DataFrame(rows, columns=colname)

df7

,neighbourhood,avg_price
0,Tremont,54.1818181818181818
1,Parkchester,73.2400000000000000
2,Brooklyn Heights,184.4750000000000000
3,Highbridge,60.1785714285714286
4,Eltingville,67.5000000000000000
...,...,...
216,Spuyten Duyvil,158.3333333333333333
217,St. Albans,80.6818181818181818
218,Tottenville,143.3333333333333333
219,Dongan Hills,82.3333333333333333


#### 8. Relation between different types of cancellation policy and rental prices


In [ ]:
cur.execute(
    """ 
      SELECT cancellation_policy, AVG(price) AS avg_price
      FROM policy 
      JOIN house_policy USING (policy_id)
      JOIN house USING (house_id)
      GROUP BY cancellation_policy
      ORDER BY avg_price DESC;


    """
)

rows = cur.fetchall()
colname = [desc[0] for desc in cur.description]

df8=pd.DataFrame(rows, columns=colname)

df8

#### 9.What is the relationship between the number of parks and rental prices within each borough?

In [115]:
cur = conn.cursor()
view4 = """
create view park_borough as
select po.borough_id, po.borough_name, count(p.park_type) as park_number 
from population po join parks p on po.borough_id = p.borough_id 
group by po.borough_id 


"""
cur.execute(view4)
conn.commit()

In [116]:
cur.execute(
    """ 
     select ht.avg_price, pb.park_number, pb.borough_name
from house_type ht join park_borough pb
on ht.borough_id = pb.borough_id 



    """
)

rows = cur.fetchall()
colname = [desc[0] for desc in cur.description]

df9=pd.DataFrame(rows, columns=colname)

df9

,avg_price,park_number,borough_name
0,174.7414802065404475,386,Manhattan
1,85.0844155844155844,160,Staten Island
2,115.4558475526217462,613,Brooklyn
3,75.1549295774647887,393,Bronx
4,90.7379367720465890,472,Queens


#### 10. What is the relationship between population and rental price within each borough?


In [119]:
cur = conn.cursor()
view4 = """
create view population_borough as
select po.borough_id, po.borough_name, po.population_2020, po.population_share_of_NYC_in_2020
from population po 
group by po.borough_id 



"""
cur.execute(view4)
conn.commit()

In [120]:
cur.execute(
    """ 
    select ht.avg_price, pob.population_2020, pob.population_share_of_NYC_in_2020, pob.borough_name
    from house_type ht join population_borough pob
    on ht.borough_id = pob.borough_id 

"""
)

rows = cur.fetchall()
colname = [desc[0] for desc in cur.description]

df10=pd.DataFrame(rows, columns=colname)

df10

,avg_price,population_2020,population_share_of_nyc_in_2020,borough_name
0,174.7414802065404475,1638281,19.16,Manhattan
1,85.0844155844155844,487155,5.7,Staten Island
2,115.4558475526217462,2648452,30.97,Brooklyn
3,75.1549295774647887,1446788,16.92,Bronx
4,90.7379367720465890,2330295,27.25,Queens


In [114]:
# close connection 
conn.close

<function connection.close>